# Importing Dependencies

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
from glob import glob
from tqdm import tqdm
import time
import tensorflow.keras.backend as K
from functools import reduce

# User Directory

In [2]:
# set to be predicted dataset path 
data_path = "./PS3_Test/seq_4/frames/"

In [3]:
# set save path directory
save_path = './PS3_Test/seq_4/save'
if not os.path.exists(save_path):
    os.mkdir(save_path)
else:
    print('Folder already exists.')

In [4]:
height = 1024//4 # Set height of your frame
width = 1280//4 # Set width of input frame

classes_and_colors = np.array([[0,0,0] ,[0,0,255], [187,155,25], 
                      [0,255,125], [255,255,125], [123,15,175],
                      [12,255,141] ])

In [5]:
# set model path for each U-NET
model_files = ['./model/updt/unet_jacc_N1.h5',
               "./model/updt/unet_jacc_N2.h5",
               './model/updt/unet_jacc_N3.h5',
               # "./model/updt/unet_jacc_N4.h5",
               "./model/updt/unet_jacc_N5.h5",]
               # "./model/updt/unet_jacc_N6.h5"]

# U-NET Model Functions

In [6]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [8]:
def dice_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true) + K.sum(y_pred)
    dice = (2. * intersection + smooth) / (sum_ + smooth)
    return 1 - dice

In [9]:
def load_model(model_path : str, jacc = 0):
    if jacc == 0:
        return tf.keras.models.load_model(model_path, custom_objects={'jaccard_distance': jaccard_distance})
    else:
        return tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss})


In [10]:
def change_color_of_binary_mask(image : list, class_id : int):
    return np.array([ [ classes_and_colors[class_id] if pixel == 255 \
            else classes_and_colors[0] for pixel in row ] for row in image ])

In [11]:
def get_class_wise_predicted_mask ( x , class_id, jacc ) :
    name = x.split('/')[-1]
    
    x = cv2.imread ( x , cv2.IMREAD_COLOR )
    x = cv2.resize ( x , ( width , height ) )
    x = x / 255.0
    x = np.expand_dims ( x , axis = 0 )
    
    model = load_model(model_files[class_id-1],jacc)
    print(model_files[class_id-1])

    p = model.predict(x)[0]
    
    p = p > 0.5
    p = p * 255

    p = np.squeeze(p,axis=-1)

    colored_mask = change_color_of_binary_mask( p ,class_id)
    
    return colored_mask.astype(np.uint8)

# Prediction

In [12]:
def concatenate_masks( first_mask, second_mask ):
    return [[ second_mask [ row_index ][ col_index ] or pixel \
            for col_index , pixel in enumerate ( row ) ]\
            for row_index , row in enumerate ( first_mask ) ]

def concatenate_masks_rgb(first_mask, second_mask):
    return [[
        [second_mask[row_index][col_index][channel] or pixel[channel]
         for channel in range(3)]  # Iterate over RGB channels
        for col_index, pixel in enumerate(row)]
        for row_index, row in enumerate(first_mask)]

def concatenate_class_wise_masks ( image_path : str, num_of_classes = 6 ):
    jacc_class = [ '',0,1,0,1 ]
    image = cv2.imread ( image_path )
    h, w, _ = image.shape

    blank_canvas = np.zeros(( 256 , 320, 3 ))

    predicted_masks = [ get_class_wise_predicted_mask( image_path, class_id, jacc = jacc_class[class_id] ) \
                       for class_id in range ( 1 , num_of_classes + 1 )]  
    
    predicted_masks.append(blank_canvas)

    #plt.imshow(predicted_masks[0])
    #plt.imshow(predicted_masks[1])
                                
    return np.array(reduce( concatenate_masks_rgb , predicted_masks ))


# Driver Code

In [13]:
for img in os.listdir(data_path):
    img_path = os.path.join(data_path, img)
    final_img = concatenate_class_wise_masks (img_path, 4)
    plt.imsave(os.path.join(save_path, f'{img}'), final_img)

print('\nDone')

./model/updt/unet_jacc_N1.h5
1/1 [==============================] - 5s 5s/step
./model/updt/unet_jacc_N2.h5
1/1 [==============================] - 0s 353ms/step
./model/updt/unet_jacc_N3.h5
1/1 [==============================] - 0s 308ms/step
./model/updt/unet_jacc_N5.h5
1/1 [==============================] - 0s 308ms/step
./model/updt/unet_jacc_N1.h5
1/1 [==============================] - 0s 321ms/step
./model/updt/unet_jacc_N2.h5
1/1 [==============================] - 0s 318ms/step
./model/updt/unet_jacc_N3.h5
1/1 [==============================] - 0s 308ms/step
./model/updt/unet_jacc_N5.h5
1/1 [==============================] - 0s 358ms/step
./model/updt/unet_jacc_N1.h5
1/1 [==============================] - 0s 338ms/step
./model/updt/unet_jacc_N2.h5
1/1 [==============================] - 0s 307ms/step
./model/updt/unet_jacc_N3.h5
1/1 [==============================] - 0s 354ms/step
./model/updt/unet_jacc_N5.h5
1/1 [==============================] - 0s 309ms/step
./model/updt/unet_j